In [1]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
import nltk
import pandas as pd

In [3]:
#Note that for this to work the NoMoreSilence_ProjectData.tsv file needs to be
#in the same directory (folder) that this notebook file is in, and that you started
#the jupyter notebook from.
df = pd.read_csv('NoMoreSilence_ProjectData.tsv', sep='\t')

In [6]:
#to show us the columns of the data we're working with
df.columns

Index(['Collection Title', 'Title', 'Local Identifier ', 'Type', 'Date ',
       'Date Type', 'Publication/Origination Info', 'Creator 1 Name',
       'Creator 1 NameType', 'Creator 1 Source', 'Creator 2 Name',
       'Creator 2 NameType', 'Creator 2 Source', 'Creator 3 Name',
       'Creator 3 NameType', 'Creator 3 Source', 'Creator 4 Name',
       'Creator 4 NameType', 'Creator 4 Source', 'Format/Physical Description',
       'Language ', 'Language Code', 'Copyright Status', 'Copyright Statement',
       'Source', 'Subject (Name) 1 Name', 'Subject (Name) 1 Name Type',
       'Subject (Name) 1 Source', 'Subject (Name) 2 Name',
       'Subject (Name) 2 Name Type', 'Subject (Name) 2 Source',
       'Subject (Name) 3 Name', 'Subject (Name) 3 Name Type',
       'Subject (Name) 3 Source', 'Subject (Topic) 1 Heading',
       'Subject (Topic) 1 Heading Type', 'Subject (Topic) 1 Source',
       'Subject (Topic) 2 Heading', 'Subject (Topic) 2 Heading Type',
       'Subject (Topic) 2 Source', '

In [8]:
#taking a look at 'Source' column as a way to potentially pull out something to categorize by collection, 
#like the call number. This looks pretty good, let's use it. 
df['Source'][0:10]

0    AIDS Legal Referral Panel Records, 2000-46, Bo...
1    AIDS Legal Referral Panel Records, 2000-46, Bo...
2    AIDS Legal Referral Panel Records, 2000-46, Bo...
3    AIDS Legal Referral Panel Records, 2000-46, Bo...
4    AIDS Legal Referral Panel Records, 2000-46, Bo...
5    AIDS Legal Referral Panel Records, 2000-46, Bo...
6    AIDS Legal Referral Panel Records, 2000-46, Bo...
7    AIDS Legal Referral Panel Records, 2000-46, Bo...
8    AIDS Legal Referral Panel Records, 2000-46, Bo...
9    AIDS Legal Referral Panel Records, 2000-46, Bo...
Name: Source, dtype: object

In [14]:
#this is creating a variable of all the sources, which we'll use to experiment with pulling out
#just the call number for each collection. 
#the for loop will iterate through each source, and use the .split method to create a new list
#with each element (separated by commas, which we specified with .split(', ') -- note 
# comma then space) as a list item.
#this threw an error, because one of the entries was a float not a string. We have to choose to 
#either make it a string or to ignore it. In this code I've made it a string with source = str(source)
#but it may actually be better to ignore it. (with an if else statement)
sources = df['Source']
for source in sources:
    source = str(source)
    s_list = source.split(', ')
    print(s_list)

['AIDS Legal Referral Panel Records', '2000-46', 'Box 3', 'Folder 2']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 4', 'Folder 4']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 7', 'Folder 2']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 1', 'Folder 10']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 3', 'Folder 6']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 5', 'Folder 4']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 4', 'Folder 8']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 1', 'Folder 1']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 3', 'Folder 13']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 6', 'Folder 1']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 3', 'Folder 10']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 1', 'Folder 6']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 7', 'Folder 1']
['AIDS Legal Referral Panel Records', '2000-46', 'Box 2', 'Folder 7']
['AIDS Legal Refe

In [15]:
#note that from the above it is the Mobilization Against AIDS records that contain the spaces as s_list[1]
#that were giving us problems.
#Additionally in People v Owens Bathhouse we find the completely empty entries, which had to be made as blank.
#also Barbara Cameron have totally blank 2nd field in 1 entry. 